# 如何在多代理应用程序中添加多轮对话（功能性API）

!!!信息“先决条件”
本指南假设您熟悉以下内容：

- [多代理系统](../../concepts/multi_agent)
- [人在循环](../../concepts/ human_in_the_loop)
- [功能 API](../../concepts/function_api)
- [命令](../../concepts/low_level/#command)
- [LangGraph 术语表](../../concepts/low_level/)


在本操作指南中，我们将构建一个应用程序，允许最终用户与一个或多个代理进行“多轮对话”。我们将创建一个使用 [`interrupt`](https://langchain-ai.github.io/langgraphjs/reference/functions/langgraph.interrupt-1.html) 的节点来收集用户输入并路由回 **active** 代理。

代理将作为工作流中的任务实现，执行代理步骤并确定下一步操作：

1. **等待用户输入**继续对话，或者
2. **通过 [**handoff**](../../concepts/multi_agent/#handoffs) 路由到另一个代理**（或返回到自身，例如在循环中）。

!!!注意兼容性

本指南需要 `@langchain/langgraph>=0.2.42` 和 `@langchain/core>=0.3.36`。

## 设置

首先，安装本示例所需的依赖项：
```bash
npm install @langchain/langgraph @langchain/anthropic @langchain/core uuid zod
```
接下来，我们需要为 Anthropic 设置 API 密钥（我们将使用的 LLM）：
```typescript
process.env.ANTHROPIC_API_KEY = "YOUR_API_KEY";
```
!!!提示“为 LangGraph 开发设置 [LangSmith](https://smith.langchain.com)”

注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 — 在[此处](https://docs.smith.langchain.com)了解有关如何开始的更多信息

在此示例中，我们将建立一个可以相互通信的旅行助理代理团队。

我们将创建 2 个代理：

* `travelAdvisor`：可以帮助推荐旅行目的地。可以向`hotelAdvisor`寻求帮助。
* `hotelAdvisor`：可以帮助推荐酒店。可以向`travelAdvisor`寻求帮助。

这是一个完全连接的网络 - 每个代理都可以与任何其他代理通信。

In [1]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";

// 获取旅行建议的工具
const getTravelRecommendations = tool(async () => {
  const destinations = ["aruba", "turks and caicos"];
  return destinations[Math.floor(Math.random() * destinations.length)];
}, {
  name: "getTravelRecommendations",
  description: "Get recommendation for travel destinations",
  schema: z.object({}),
});

// 获取酒店推荐的工具
const getHotelRecommendations = tool(async (input: { location: "aruba" | "turks and caicos" }) => {
  const recommendations = {
    "aruba": [
      "The Ritz-Carlton, Aruba (Palm Beach)",
      "Bucuti & Tara Beach Resort (Eagle Beach)"
    ],
    "turks and caicos": ["Grace Bay Club", "COMO Parrot Cay"]
  };
  return recommendations[input.location];
}, {
  name: "getHotelRecommendations",
  description: "Get hotel recommendations for a given destination.",
  schema: z.object({
    location: z.enum(["aruba", "turks and caicos"])
  }),
});

// 定义一个工具来发出移交给不同代理的意图
// 注意：这不是使用 Command(goto) 语法导航到不同的代理：
// 下面的“workflow()”显式处理切换
const transferToHotelAdvisor = tool(async () => {
  return "Successfully transferred to hotel advisor";
}, {
  name: "transferToHotelAdvisor",
  description: "Ask hotel advisor agent for help.",
  schema: z.object({}),
  // 提示我们的代理实施应该停止
  // 调用此工具后立即
  returnDirect: true,
}); 

const transferToTravelAdvisor = tool(async () => {
  return "Successfully transferred to travel advisor";
}, {
  name: "transferToTravelAdvisor", 
  description: "Ask travel advisor agent for help.",
  schema: z.object({}),
  // 提示我们的代理实施应该停止
  // 调用此工具后立即
  returnDirect: true,
});

!!!注意“传输工具”

您可能已经注意到，我们在传输工具中使用了 `tool(... { returnDirect: true })`。这样做是为了在调用这些工具后，各个代理（例如，`travelAdvisor`）可以尽早退出 ReAct 循环，而无需最后一次调用模型来处理工具调用的结果。这是期望的行为，因为我们希望检测代理何时调用此工具并将控制权“立即”移交给不同的代理。

**注意**：这意味着与预构建的 [`createReactAgent`](/langgraphjs/reference/functions/langgraph_prebuilt.createReactAgent.html) 一起使用 - 如果您正在构建自定义代理，请确保手动添加逻辑以处理标记为 `returnDirect` 的工具的提前退出。

现在让我们使用预构建的 [`createReactAgent`](/langgraphjs/reference/functions/langgraph_prebuilt.createReactAgent.html) 和我们的多代理工作流程来创建代理。请注意，每次我们收到每个代理的最终响应后，都会调用 [`interrupt`](/langgraphjs/reference/functions/langgraph.interrupt-1.html) 。

In [3]:
import {
  AIMessage,
  type BaseMessage,
  type BaseMessageLike
} from "@langchain/core/messages";
import { ChatAnthropic } from "@langchain/anthropic";
import { createReactAgent } from "@langchain/langgraph/prebuilt";
import {
  addMessages,
  entrypoint,
  task,
  MemorySaver,
  interrupt,
} from "@langchain/langgraph";

const model = new ChatAnthropic({
  model: "claude-3-5-sonnet-latest",
});

const travelAdvisorTools = [
  getTravelRecommendations,
  transferToHotelAdvisor,
];

// 定义旅行顾问 ReAct 代理
const travelAdvisor = createReactAgent({
  llm: model,
  tools: travelAdvisorTools,
  stateModifier: [
    "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc).",
    "If you need hotel recommendations, ask 'hotel_advisor' for help.",
    "You MUST include human-readable response before transferring to another agent.",
  ].join(" "),
});

// 您还可以添加其他逻辑，例如更改代理的输入/代理的输出等。
// 注意：我们正在使用状态中消息的完整历史记录调用 ReAct 代理
const callTravelAdvisor = task("callTravelAdvisor", async (messages: BaseMessageLike[]) => {
  const response = await travelAdvisor.invoke({ messages });
  return response.messages;
});

const hotelAdvisorTools = [
  getHotelRecommendations,
  transferToTravelAdvisor,
];

// 定义酒店顾问 ReAct 代理
const hotelAdvisor = createReactAgent({
  llm: model,
  tools: hotelAdvisorTools,
  stateModifier: [
    "You are a hotel expert that can provide hotel recommendations for a given destination.",
    "If you need help picking travel destinations, ask 'travel_advisor' for help.",
    "You MUST include a human-readable response before transferring to another agent."
  ].join(" "),
});

// 添加酒店顾问任务
const callHotelAdvisor = task("callHotelAdvisor", async (messages: BaseMessageLike[]) => {
  const response = await hotelAdvisor.invoke({ messages });
  return response.messages;
});

const checkpointer = new MemorySaver();

const multiTurnGraph = entrypoint({
  name: "multiTurnGraph",
  checkpointer,
}, async (messages: BaseMessageLike[]) => {  
  let callActiveAgent = callTravelAdvisor;
  let agentMessages: BaseMessage[];
  let currentMessages = messages;
  while (true) {
    agentMessages = await callActiveAgent(currentMessages);
    
    // 查找最后一条 AI 消息
    // 如果调用其中一个切换工具，则返回最后一条消息
    // 代理将是 ToolMessages 因为我们将它们设置为
    // “returnDirect：true”。这意味着最后一条 AIMessage 将
    // 有工具调用。
    // 否则，最后返回的消息将是一条 AIMessage
    // 没有工具调用，这意味着我们已经准备好接受新的输入。
    const reversedMessages = [...agentMessages].reverse();
    const aiMsgIndex = reversedMessages
      .findIndex((m): m is AIMessage => m.getType() === "ai");
      
    const aiMsg: AIMessage = reversedMessages[aiMsgIndex];
  
    // 我们将最后一条 AI 消息之前的所有消息附加到当前消息中。
    // 这可能包括 ToolMessages（如果调用了切换工具）
    const messagesToAdd = reversedMessages.slice(0, aiMsgIndex + 1).reverse();

    // 添加代理的回复
    currentMessages = addMessages(currentMessages, messagesToAdd);

    if (!aiMsg?.tool_calls?.length) {
      const userInput = await interrupt("Ready for user input.");
      if (typeof userInput !== "string") {
        throw new Error("User input must be a string.");
      }
      if (userInput.toLowerCase() === "done") {
        break;
      }
      currentMessages = addMessages(currentMessages, [{
        role: "human",
        content: userInput,
      }]);
      continue;
    }

    const toolCall = aiMsg.tool_calls.at(-1)!;
    if (toolCall.name === "transferToHotelAdvisor") {
      callActiveAgent = callHotelAdvisor;
    } else if (toolCall.name === "transferToTravelAdvisor") {
      callActiveAgent = callTravelAdvisor;
    } else {
      throw new Error(`Expected transfer tool, got '${toolCall.name}'`);
    }
  }

  return entrypoint.final({
    value: agentMessages[agentMessages.length - 1],
    save: currentMessages,
  });
});

我们使用 while 循环来实现代理和用户之间的连续对话。该循环允许：

1. 获取代理响应
2. 处理代理人之间的转账
3.通过中断收集用户输入
4. 恢复使用特殊输入（参见下面的`Command`）

## 测试多轮对话

让我们用这个应用程序测试一次多轮对话。

In [4]:
import { v4 as uuidv4 } from 'uuid';
import { Command } from "@langchain/langgraph";
import { isBaseMessage } from "@langchain/core/messages";

const threadConfig = {
  configurable: { 
    thread_id: uuidv4() 
  },
  streamMode: "updates" as const,
};

const inputs = [
  // 第一轮对话
  [{ role: "user", content: "i wanna go somewhere warm in the caribbean" }],
  // 由于我们正在使用“中断”，因此我们需要恢复使用命令原语
  // 第二轮对话
  new Command({
    resume: "could you recommend a nice hotel in one of the areas and tell me which area it is."
  }),
  // 第三轮对话
  new Command({
    resume: "i like the first one. could you recommend something to do near the hotel?"
  })
];

const runConversation = async () => {
  for (const [idx, userInput] of inputs.entries()) {
    console.log();
    console.log(`--- Conversation Turn ${idx + 1} ---`);
    console.log();
    console.log(`User: ${JSON.stringify(userInput, null, 2)}`);
    console.log();
    
    const stream = await multiTurnGraph.stream(
      userInput as any,
      threadConfig,
    );

    for await (const update of stream) {
      if (update.__metadata__?.cached) {
        continue;
      }
      for (const [nodeId, value] of Object.entries(update)) {
        if (Array.isArray(value) && value.length > 0) {
          const lastMessage = value.at(-1);
          if (isBaseMessage(lastMessage) && lastMessage?.getType() === "ai") {
            console.log(`${nodeId}: ${lastMessage.content}`);
          }
        }
      }
    }
  }
};

// 执行对话
try {
  await runConversation();
} catch (e) {
  console.error(e);
}


--- Conversation Turn 1 ---

User: [
  {
    "role": "user",
    "content": "i wanna go somewhere warm in the caribbean"
  }
]

callTravelAdvisor: Based on the recommendations, Turks and Caicos would be an excellent choice for your Caribbean getaway! This British Overseas Territory is known for its stunning white-sand beaches, crystal-clear turquoise waters, and year-round warm weather. Grace Bay Beach in Providenciales is consistently rated as one of the world's best beaches.

You can enjoy:
- World-class snorkeling and diving
- Luxury resorts and spas
- Fresh seafood cuisine
- Water sports like kayaking and paddleboarding
- Beautiful coral reefs
- Average temperatures between 75-85°F (24-29°C) year-round

Would you like me to connect you with our hotel advisor to help you find the perfect place to stay in Turks and Caicos?

--- Conversation Turn 2 ---

User: {
  "resume": "could you recommend a nice hotel in one of the areas and tell me which area it is.",
  "goto": []
}

callHotelA